In [1]:
import csv
import numpy as np
from collections import defaultdict
from matplotlib import pyplot
import pandas

In [2]:
BLANKS=["","RBNF1", "RBNF2", "RBNF", "0", "RBNF 1", "RBNF 2"]
#TOTAL= 5048
TOTAL=2734
#TOTAL=1351
#TOTAL=1201
#TOTAL=1168

In [3]:
def loadCsvData(fileName):
	matrix = []
	# open a file
	with open(fileName) as f:
		reader = csv.reader(f)
        # loop over each row in the file
		for row in reader:

			# cast each value to a float
			doubleRow = []
			for value in row:
				doubleRow.append(value.strip())

			# store the row into our matrix
			matrix.append(doubleRow)
	return matrix

In [4]:
data= loadCsvData("data/metadata.csv")

In [5]:
data=pandas.DataFrame(data[1:], columns=data[0])

In [6]:
interviewtxt =loadCsvData("data/interview text file to ids.csv")

In [7]:
interviewtxt=pandas.DataFrame(interviewtxt[1:], columns=interviewtxt[0])


In [8]:
text_to_ids=defaultdict(lambda: defaultdict(lambda:"0"))
for i in range(len(interviewtxt)):
    if interviewtxt["no_transcript"][i]=="TRUE":
        continue
    name=interviewtxt.at[i, interviewtxt.columns[0]]
    name=name[:name.rfind('.')]
    name=name.replace('\'','_')
    text_to_ids[name]["id"]=interviewtxt["interviewee_ids"][i].split(';')
    date=interviewtxt["date_of_first_interview"][i]
    text_to_ids[name]["interview year"]= date[date.rfind("/")+1:]

In [31]:
ids_to_years=defaultdict(lambda: defaultdict(lambda:"0"))
for i in range(len(interviewtxt)):
    date=interviewtxt["date_of_first_interview"][i]
    year=date[date.rfind('/')+1:]
    ids=interviewtxt["interviewee_ids"][i].split(';')
    no_transcript=interviewtxt["no_transcript"][i]
    for num in ids:
        ids_to_years[num]["year"]= str(year)
        if no_transcript=="TRUE" and ids_to_years[num]["included"]=="0":
            ids_to_years[num]["included"]=False
        else:
            ids_to_years[num]["included"]=True

In [32]:
database=defaultdict(lambda: defaultdict(lambda:0))
for i in range(len(data)):
    if ids_to_years[data.loc[i, "interviewee_id"]]["included"]==False:
        continue
    for atr in data.columns:
        if atr== "researcher_assumed_birth_decade":
            continue
        if atr== "researcher_assumed_race":
            continue
        value= data[atr][i]
        if value in BLANKS and atr=="\ufeffbirth_decade":
            value= data.loc[i, "researcher_assumed_birth_decade"]
        if value in BLANKS and atr=="identified_race":
            value= data.loc[i, "researcher_assumed_race"]
        if value in BLANKS and atr == "real_interviewee_birth_year":
            value= data.loc[i, "approximate_interviewee_birth_year"]
        database[data.loc[i, "interviewee_id"]][atr]=value
    database[data.loc[i, "interviewee_id"]]["interview_year"]=ids_to_years[data.loc[i, "interviewee_id"]]["year"]

In [17]:
myfile = open("data/Personal_experience_intersect_sexual_assault.txt", encoding="latin1")
txt = myfile.read()
myfile.close()

In [23]:
sectioned=[]
string=""
for i in txt:
    if i == '<':
        if len(string)>0:
            sectioned.append(string)
            string=""
    else:
        string+=i

In [39]:
more_sectioned=[(i[i.rfind('\\\\')+2:i.rfind('>')], i[i.rfind('>'):]) for i in sectioned]

In [40]:
print(more_sectioned[:10])

[('BWOH_Edmonds_Florence', "> - ¤ 1 reference coded  [0.30% Coverage]\n\nReference 1 - 0.30% Coverage\n\nFJE: Well, I might tell you this, too, with your home nursing, you were never molested, going into these homes, at all. And I always felt that our bag and uniform was our passport. You hear of so much molesting these days in New York City and in Harlem and in that area, but there was never anything of that type done to nurses. REH: That's nice. So you felt safe going anywhere. FJE: Oh, you felt safe, you never gave it a thought.\n\n"), ('BWOH_Harrison_Pleasant', '> - ¤ 3 references coded  [1.05% Coverage]\n\nReference 1 - 0.28% Coverage\n\n PH: Yes. Uncle Hector was my father\'s uncle, which was my granduncle. And I was living with my father\'s sister. But before I went to live with her, I was living with another one of my father\'s sisters, for a short period of time. And the man that she was living with wanted to molest meÌ¢åÛåÓhe wanted to use me. And my father\'s sister resented

In [53]:
for i in more_sectioned[:100]:
    database[text_to_ids[i[0]]["id"][0]]["real_interviewee_birth_year"]

In [54]:
more_sectioned.sort(key=lambda i: database[text_to_ids[i[0]]["id"][0]]["real_interviewee_birth_year"])

In [55]:
print(more_sectioned)

[('SOOH_Hall_Heinz_Hall_Marliese', '> - ¤ 1 reference coded  [4.35% Coverage]\n\nReference 1 - 4.35% Coverage\n\n We have so many stories to tell. Toward the end of the war, our families had very little food in Germany. We had to trade tablecloths and goods, things for my mother, for potatoes and things like that. We went on farms just so we could eat. I have a lot of things to tell but I usually like to keep things inside and not talk about it. You kind of don t want to think about those things anymore. He has a funny story to tell. Which one? The one about the egg in France. When you stole an egg, remember? No, not eggs, they were peaches We went into an orchard; there were fifty German airmen in an orchard. This French guy came along with a pitchfork and he chased fifty soldiers out of a peach orchard We were all kids, seventeen, eighteen years old. 16 M. Hall Wilson H. Hall Wilson H. Hall Wilson H. Hall Wilson H. Hall Wilson M. Hall Yes, I could tell you about when we got occupied 

In [56]:
for i in more_sectioned:
    print(database[text_to_ids[i[0]]["id"][0]]["real_interviewee_birth_year"])


1885
1890
1898
1902
1904
1906
1907
1915
1919
1920
1928
1928
1932
1933
1934
1936
1938
1938
1940
1941
1942
1943
1943
1944
1944
1944
1944
1945
1946
1947
1947
1948
1948
1949
1949
1949
1949
1949
1950
1950
1951
1951
1951
1951
1953
1954
1954
1955
1957
1957
1958
1958
1963
1964
1964
1965
1971
1973
1980
h


In [58]:
f = open("Sexual_assault_personal_experience.txt", "a")
for i in more_sectioned:
    f.write(i[0])
    f.write(i[1][6:])
f.close()